# Sklearn Pipelines Exercise
*Made by viga@itu.dk and thso@itu.dk*

## Introduction

In this exercise you'll be working with the [Wine Quality Dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) from the UCI Machine Learning Repository. The dataset consists of 11 features and a quality score for 4898 white wine samples and 1599 red wine samples. The goal is to predict the quality of the wine based on the features.

The datasets are located in the `data` folder. The `winequality-red.csv` file contains the red wine samples and the `winequality-white.csv` file contains the white wine samples. Lastly, the `winequality.names` file contains a description of the dataset.

The goal of this exercise is to get you familiar with the [Scikit-learn Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) API. You'll be using pipelines to perform feature scaling and feature selection.

## Load in the data

You can either load the red-wine dataset or the white-wine dataset. You can also load both datasets and combine them if you want.

Both datasets are available in the `data` folder, and are called `winequality-red.csv` and `winequality-white.csv`.

Hint: You can use the `pd.read_csv()` function to load in the data (remember to check the delimiter!). You can find the documentation [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).


In [1]:
import pandas as pd
# load in the data
dfRed=pd.read_csv('data/winequality-red.csv',sep=';')
dfWhite=pd.read_csv('data/winequality-white.csv',sep=';')
# if you want to you can combine the two datasets into one - but this is not necessary
#nah
# check a few rows of the data - hint: use .head()
dfRed.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


## Data Exploration

### Check the number of missing values in the dataset.

Hint: `.isnull()`

Dont worry if there are missing values, we'll handle them later in our pipeline!

In [2]:
dfRed.isnull()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,False,False,False,False,False,False,False,False,False,False,False,False
1595,False,False,False,False,False,False,False,False,False,False,False,False
1596,False,False,False,False,False,False,False,False,False,False,False,False
1597,False,False,False,False,False,False,False,False,False,False,False,False


### Check some basic statistics

We want to know the mean, standard deviation, minimum, maximum and quartiles of each feature.
This will give us a good idea of the distribution of the data, and also tell us if we need to do any scaling.

Hint: `.describe()`, If the output is hard to read, you can use `.T` to transpose the dataframe, i.e., swapping the rows and columns.

Do you notice anything strange about the data? Is there anything that stands out to you?

In [3]:
# check some basic statistics
dfRed.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,1599.0,8.319637,1.741096,4.60000,7.1000,7.90000,9.20000,15.90000
volatile acidity,1597.0,0.527583,0.178980,0.12000,0.3900,0.52000,0.64000,1.58000
citric acid,1596.0,0.270495,0.194613,0.00000,0.0900,0.26000,0.42000,1.00000
residual sugar,1599.0,2.538806,1.409928,0.90000,1.9000,2.20000,2.60000,15.50000
chlorides,1598.0,0.087466,0.047080,0.01200,0.0700,0.07900,0.09000,0.61100
free sulfur dioxide,1596.0,15.879073,10.469052,1.00000,7.0000,14.00000,21.00000,72.00000
total sulfur dioxide,1597.0,46.405761,32.817550,6.00000,22.0000,38.00000,62.00000,289.00000
density,1597.0,0.996748,0.001886,0.99007,0.9956,0.99675,0.99783,1.00369
pH,1597.0,3.310971,0.154418,2.74000,3.2100,3.31000,3.40000,4.01000
sulphates,1596.0,0.658164,0.169647,0.33000,0.5500,0.62000,0.73000,2.00000


We saw that there were some missing values in the dataset, this we can fix in the pipeline, using the [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) from sklearn.

Next we also saw that there was a some differences in the scale of the different variables, so we will use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from sklearn to scale the data. This will make it easier for the model to learn the patterns in the data. Especially for the KNN algorithm (which we'll use), which is based on distance, it is important that the data is scaled.

If you think of other transformations that might be useful for this dataset, feel free to try them out!

**Take a look at the [sklearn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html) module for some inspiration.**

# Splitting the data

Now that we have created our pipeline, we can train the model.

First we need to split the data into a training set and a test set. We will use the [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function from sklearn to do this. But first we need to split the data into features and labels.

The features are all the columns in the dataset, except for the `quality` column, which are the labels.

We will use the default split of 75% training data, and 25% test data.

Hint: You can use the `random_state` parameter to make sure that the data is split the same way every time you run the code.

The train_test_split function returns four values, the first two are the training and test data, and the last two are the train and test labels.

In [19]:
from sklearn.model_selection import train_test_split
# split the data into X and y
print(dfRed.columns)
XRed=dfRed[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']]
yRed=dfRed['quality']
# now split the data into train and test data
xTrain,xTest,yTrain,yTest=train_test_split(XRed,yRed,random_state=42)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')


# Creating the pipeline

We will now create a pipeline that will handle the missing values and scaling for us, and finally train a KNN model on the data.

We will use the [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class from sklearn to create our pipeline.

The pipeline will consist of three steps, the first step will be to impute the missing values, and the second step will be to scale the data, and the third step will be to train the model.

The pipeline format is a list of tuples, where the first element in the tuple is the name of the step, and the second element is the step itself, e.g.:

```python
pipeline = Pipeline([
	('step_name', step()),
	('step_name', step()),
	('step_name', step()),
])
```

Where the `step_name` is a string, and the `step` is a sklearn object - this can be a "Transformer" object (like `SimpleImputer` and `StandardScaler`) or an "Estimator" object (like `KNeighborsClassifier` or `LinearRegression`).

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

# create your pipeline
pipe = Pipeline([
    ('impute',SimpleImputer()),
    ('scale',StandardScaler()),
    ('knn',KNeighborsClassifier()),
])

# Evaluating the model

Now that we have trained the model, we want to evaluate it to see how well it performs.

We will use the [accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) function from sklearn to calculate the accuracy of the model.

Since we have created a pipeline, we can simply call the `.fit()` and `.predict()` methods on the pipeline object, and it will handle the preprocessing for us - and importantly in the correct order.

Remember to only call `.fit()` on the training data. Calling `.fit()` on the test data will cause the model to overfit to the test data, and will give you an overly optimistic accuracy score.

* **`.fit(X_train, y_train)` will train the model on the training data.**
* **`.predict(X_train)` will return the predicted labels for the test data, which you can then pass to the `accuracy_score` function, along with the true labels (y_train).**
* **`.predict(X_test)` will return the predicted labels for the test data, which you can then pass to the `accuracy_score` function, along with the true labels (y_test).**

In [24]:
from sklearn.metrics import accuracy_score
# fit the pipeline
pipe.fit(xTrain,yTrain)

# evaluate the pipeline
print('Train accuracy',accuracy_score(yTrain,pipe.predict(xTrain)))
print('Test accuracy',accuracy_score(yTest,pipe.predict(xTest)))

Train accuracy 0.7114261884904087
Test accuracy 0.59
